In [1]:
%load_ext autoreload
%autoreload 


In [2]:
import timeit
import copy
import concurrent.futures 
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import batoid
import ghosts.plotter as plotter
import ghosts.simulator as simulator
import ghosts.tweak_optics as tweak_optics

from ghosts.analysis import compute_ghost_separations, reduce_ghosts
from ghosts.analysis import match_ghosts, compute_reduced_distance, compute_2d_reduced_distance, find_nearest_ghost
from ghosts.analysis import make_data_frame

from ghosts.beam_configs import BEAM_CONFIG_1, BEAM_CONFIG_2, FAST_BEAM_CONFIG_1



In [3]:
from ghosts.constants import *
print(f'LSST Camera has {LSST_CAMERA_PIXEL_DENSITY_MM2:.0f} pixels per mm^2')

LSST Camera has 36000 pixels per mm^2


In [4]:
# CCOB like geometry, i.e. lenses + filters
telescope = batoid.Optic.fromYaml("../data/LSST_CCOB_r.yaml")

In [5]:
# test list of optics
assert tweak_optics.get_list_of_optics(telescope)==['L1', 'L2', 'Filter', 'L3', 'Detector'], 'Not a CCOB optical setup'

# Make refractive interfaces partially reflective
# Call on current telescope
tweak_optics.make_optics_reflective(telescope)


0.02

In [6]:
# Run simulation with standard beam config
traceFull, rForward, rReverse, rays = simulator.run_simulation(telescope, beam_config=BEAM_CONFIG_1)


In [7]:
# Analyze
spots_data, _spots = reduce_ghosts(rForward)
data_frame = make_data_frame(spots_data, BEAM_CONFIG_1)


In [8]:
data_frame

,config,n_photons,beam_x,beam_y,beam_width,beam_theta,beam_phi,index,name,pos_x,std_x,pos_y,std_y,width_x,width_y,radius,radius_err,surface,pixel_signal
0,1,1000,0.1,0,0.00125,0.0,0.0,0,"(L1_exit, L1_entrance)",0.044590,0.000280,0.000016,0.000280,0.001105,0.001106,0.001105,6.018804e-07,3.838238,516.383687
1,1,1000,0.1,0,0.00125,0.0,0.0,1,"(L2_exit, L1_entrance)",0.142045,0.000892,0.000051,0.000892,0.003515,0.003523,0.003519,4.098390e-06,38.902118,46.993244
2,1,1000,0.1,0,0.00125,0.0,0.0,2,"(L2_exit, L1_exit)",0.218109,0.001377,0.000078,0.001377,0.005426,0.005410,0.005418,8.256835e-06,92.215057,20.642138
3,1,1000,0.1,0,0.00125,0.0,0.0,3,"(L2_entrance, L1_entrance)",0.003179,0.000018,0.000001,0.000018,0.000072,0.000079,0.000076,3.327172e-06,0.017917,106243.110864
4,1,1000,0.1,0,0.00125,0.0,0.0,4,"(L2_entrance, L1_exit)",0.055177,0.000347,0.000020,0.000347,0.001367,0.001369,0.001368,8.985820e-07,5.876024,337.303487
5,1,1000,0.1,0,0.00125,0.0,0.0,5,"(L2_exit, L2_entrance)",0.165043,0.001050,0.000059,0.001050,0.004136,0.004093,0.004114,2.106976e-05,53.183983,37.266921
6,1,1000,0.1,0,0.00125,0.0,0.0,6,"(Filter_exit, L1_entrance)",0.111794,0.000703,0.000040,0.000703,0.002768,0.002773,0.002771,2.145156e-06,24.115463,69.922567
7,1,1000,0.1,0,0.00125,0.0,0.0,7,"(Filter_exit, L1_exit)",0.195100,0.001225,0.000070,0.001225,0.004828,0.004839,0.004833,5.592693e-06,73.391518,23.922962
8,1,1000,0.1,0,0.00125,0.0,0.0,8,"(Filter_exit, L2_entrance)",0.175419,0.001110,0.000063,0.001110,0.004372,0.004351,0.004361,1.044467e-05,59.753991,30.594387
9,1,1000,0.1,0,0.00125,0.0,0.0,9,"(Filter_exit, L2_exit)",0.096187,0.000605,0.000034,0.000605,0.002384,0.002386,0.002385,8.125699e-07,17.867934,106.532517


In [9]:
BEAM_CONFIG_1

{'id': 1,
 'wl': 5e-07,
 'n_photons': 1000,
 'x_offset': 0.1,
 'y_offset': 0,
 'radius': 0.00125,
 'theta': 0.0,
 'phi': 0.0}

In [10]:
GEOM_CONFIG_0 = {'geom_id': 0,
                 'L1': {'shift': [0.0, 0.0, 0.0], 'rotations': [0., 0., 0.]},
                 'L2': {'shift': [0.0, 0.0, 0.0], 'rotations': [0., 0., 0.]},
                 'L3': {'shift': [0.0, 0.0, 0.0], 'rotations': [0., 0., 0.]},
                 'Filter': {'shift': [0.0, 0.0, 0.0], 'rotations': [0., 0., 0.]},
                 'Detector': {'shift': [0.0, 0.0, 0.0], 'rotations': [0., 0., 0.]}}

In [11]:
gd0 = pd.DataFrame(data=GEOM_CONFIG_0)
gd0

,geom_id,L1,L2,L3,Filter,Detector
shift,0,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
rotations,0,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"


In [12]:
GEOM_CONFIG_100 = {'geom_id': 100, 'L1': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
                          'L2': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
                          'L3': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]}}

In [13]:
GEOM_CONFIG_1 = {'geom_id': 1, 'L2' : {'rotations': [0.1, 0., 0.]}}

In [14]:
gd = pd.DataFrame(data=GEOM_CONFIG_100)
gd

,geom_id,L1,L2,L3
shift,100,"[0.001, 0.001, 0.001]","[0.001, 0.001, 0.001]","[0.001, 0.001, 0.001]"
rotations,100,"[0.1, 0.1, 0.1]","[0.1, 0.1, 0.1]","[0.1, 0.1, 0.1]"


In [15]:
gd2 = pd.DataFrame(data=GEOM_CONFIG_1)
gd2

,geom_id,L2
rotations,1,"[0.1, 0.0, 0.0]"


In [16]:
gc = pd.concat([gd0, gd, gd2])
gc.sort_values('geom_id')

,geom_id,L1,L2,L3,Filter,Detector
shift,0,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
rotations,0,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
rotations,1,NaN,"[0.1, 0.0, 0.0]",NaN,NaN,NaN
shift,100,"[0.001, 0.001, 0.001]","[0.001, 0.001, 0.001]","[0.001, 0.001, 0.001]",NaN,NaN
rotations,100,"[0.1, 0.1, 0.1]","[0.1, 0.1, 0.1]","[0.1, 0.1, 0.1]",NaN,NaN


In [17]:
newf=gc[gc['geom_id']==100]
newf


,geom_id,L1,L2,L3,Filter,Detector
shift,100,"[0.001, 0.001, 0.001]","[0.001, 0.001, 0.001]","[0.001, 0.001, 0.001]",NaN,NaN
rotations,100,"[0.1, 0.1, 0.1]","[0.1, 0.1, 0.1]","[0.1, 0.1, 0.1]",NaN,NaN


In [31]:
# get shifts dict way
gc[gc['geom_id']==100]['L1']['shift']

[0.001, 0.001, 0.001]

In [36]:
for key, val in gc[gc['geom_id']==100].items():
    print(val['rotations'])

100
[0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
nan
nan


In [18]:
h=newf.to_dict()
h

{'geom_id': {'shift': 100, 'rotations': 100},
 'L1': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
 'L2': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
 'L3': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
 'Filter': {'shift': nan, 'rotations': nan},
 'Detector': {'shift': nan, 'rotations': nan}}

In [19]:
h['geom_id']=h['geom_id']['shift']
h

{'geom_id': 100,
 'L1': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
 'L2': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
 'L3': {'shift': [0.001, 0.001, 0.001], 'rotations': [0.1, 0.1, 0.1]},
 'Filter': {'shift': nan, 'rotations': nan},
 'Detector': {'shift': nan, 'rotations': nan}}

In [20]:
newf2=gc[gc['geom_id']==1]
print(newf2)
newf2.to_dict()

           geom_id   L1               L2   L3 Filter Detector
rotations        1  NaN  [0.1, 0.0, 0.0]  NaN    NaN      NaN


{'geom_id': {'rotations': 1},
 'L1': {'rotations': nan},
 'L2': {'rotations': [0.1, 0.0, 0.0]},
 'L3': {'rotations': nan},
 'Filter': {'rotations': nan},
 'Detector': {'rotations': nan}}

In [27]:
# Alternative Structure
GEOM2_CONFIG_100 = {'L1': [[0.002, 0.001, 0.001], [0.2, 0.1, 0.1]],
                    'L2': [[0.002, 0.001, 0.001], [0.2, 0.1, 0.1]],
                    'L3': [[0.002, 0.001, 0.001], [0.2, 0.1, 0.1]]}

In [22]:
gd2 = pd.DataFrame(data=GEOM2_CONFIG_100)
gd2


,L1,L2,L3
0,"[0.002, 0.001, 0.001]","[0.002, 0.001, 0.001]","[0.002, 0.001, 0.001]"
1,"[0.2, 0.1, 0.1]","[0.2, 0.1, 0.1]","[0.2, 0.1, 0.1]"


In [23]:
GEOM3_CONFIG_100 = {'L1': [0.002, 0.001, 0.001, 0.2, 0.1, 0.1],
                    'L2': [0.002, 0.001, 0.001, 0.2, 0.1, 0.1],
                    'L3': [0.002, 0.001, 0.001, 0.2, 0.1, 0.1]}

In [24]:
gcd = pd.DataFrame(
    {
        "L1": np.array(GEOM3_CONFIG_100['L1'], dtype="float"),
        "L2": np.array(GEOM3_CONFIG_100['L2'], dtype="float"),
        "L3": np.array(GEOM3_CONFIG_100['L3'], dtype="float"),
    }
)


In [25]:
gcd

,L1,L2,L3
0,0.002,0.002,0.002
1,0.001,0.001,0.001
2,0.001,0.001,0.001
3,0.200,0.200,0.200
4,0.100,0.100,0.100
5,0.100,0.100,0.100
